In [1]:
import open3d as o3d
import numpy as np
import random
from sklearn.neighbors import NearestNeighbors
import pandas as pd
from sklearn.manifold import MDS
from matplotlib import pyplot as plt
import os


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# Import the coordinates and the features that have been generated by dMaSIF-site of the following proteins
#   - Subunit A (IgG Fc)
#   - Subunit C (GB1 protein)
#   - Complex (Subunit A and C combined)

%cd -q masif_site_outputs
coords_A = np.load("1fcc_A_predcoords.npy")
features_A = np.load("1fcc_A_predfeatures_emb1.npy")

coords_C = np.load("1fcc_C_predcoords.npy")
features_C = np.load("1fcc_C_predfeatures_emb1.npy")

coords_complex = np.load("1fcc_A, C_predcoords.npy")
features_complex = np.load("1fcc_A, C_predfeatures_emb1.npy")

In [3]:
coords_A.shape, coords_C.shape, coords_complex.shape

((25600, 3), (7317, 3), (31156, 3))

In [4]:
#create a PointCloud and add the points

pointcloud_A = o3d.geometry.PointCloud()
pointcloud_A.points = o3d.utility.Vector3dVector(coords_A)
o3d.visualization.draw_geometries([pointcloud_A])

pointcloud_C = o3d.geometry.PointCloud()
pointcloud_C.points = o3d.utility.Vector3dVector(coords_C)
#o3d.visualization.draw_geometries([pointcloud_C])

pointcloud_complex = o3d.geometry.PointCloud()
pointcloud_complex.points = o3d.utility.Vector3dVector(coords_complex)
#o3d.visualization.draw_geometries([pointcloud_complex])

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 


In [5]:
# Create color vector for pointclouds
blue = [0, 0.5, 1]
red = [1, 0, 0]
colors_A = np.zeros((len(coords_A),3))
colors_C = np.zeros((len(coords_C),3))
colors_complex = np.zeros((len(coords_complex),3))
colors_A[:] = blue
colors_C[:] = blue
colors_complex = blue

In [6]:
def find_buried_points(subunit_coords, complex_coords, threshold):

    '''Function that takes as input the coordinates of two similar pointclouds (one subunit and
    one complex involving that subunit) and a threshold. Returns a list of indeces indicating 
    which points of the first pointcloud (subunit) are more than a certain distance (threshold)
    away from any point in the second pointcloud. These points are "buried" in the complex '''

    neigh = NearestNeighbors(n_neighbors=1)
    neigh.fit(coords_complex) 

    buried= []
    for i in range(len(subunit_coords)):
        if neigh.kneighbors([subunit_coords[i]])[0][0][0] > threshold:
            buried.append(i)
    return buried

In [7]:
# Color pointcloud points according to whether they are buried (red) or not (blue)

colors_A[find_buried_points(coords_A, coords_complex, 2)]=red
colors_C[find_buried_points(coords_C, coords_complex, 2)]=red

pointcloud_A.colors = o3d.utility.Vector3dVector(colors_A)
pointcloud_C.colors = o3d.utility.Vector3dVector(colors_C)

In [8]:
o3d.visualization.draw_geometries([pointcloud_C])

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 


In [9]:
o3d.visualization.draw_geometries([pointcloud_A])